TEXT Pre-Processing

In [4]:
#install yang dibutuhkan
!pip install unidecode
!pip install demoji
!pip install contractions
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fefdd79ee8a1be4a56fb2354c590d73e832206f4df9268b28b5a3d14cb22ab91
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
#import library yang dibutuhkan
import numpy as np
import pandas as pd
import nltk
import spacy
import re
import string
import unidecode
import demoji
import contractions
from num2words import num2words
from nltk.corpus import twitter_samples
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [6]:
#setting awal
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

nltk.download('twitter_samples') #untuk unduh dataset twitter kita

#ambil twit/cuit positif dan negatif
for name in twitter_samples.fileids():
  print(f'-{name}')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


-negative_tweets.json
-positive_tweets.json
-tweets.20150430-223406.json


In [16]:
#ini yang negative kita kasih tanda 0
negative_tweets = twitter_samples.strings("negative_tweets.json")
df_neg = pd.DataFrame(negative_tweets, columns=['text'])
df_neg['label'] = 0

#ini yang positif kita kasih tanda 1
positive_tweets = twitter_samples.strings("positive_tweets.json")
df_pos = pd.DataFrame(positive_tweets, columns=['text'])
df_pos['label'] = 1

df = pd.concat([df_pos, df_neg]) #menggabungkan menjadi 1 dataframe
df = df.sample(frac=1).reset_index(drop=True) #mencampur agar acak

In [11]:
df.head()

,text,label
0,@clarelea101 At least it's Friday :D,1
1,it was a really good stream @radiosilents! thank you! :),1
2,@Jesutomisin I hear you guys want Arbeloa. :D,1
3,@RblSports upgraded ans synced up. Plus it can be done from a single remote device. Getting there folks. :),1
4,@irisharry_ Awww no :( I didn't mean to make you sad! - Mike,0


#1. Lower Case = jadikan semua huruf kecil!!!!!!

In [17]:
df.text = df.text.str.lower()
df.head()

,text,label
0,please go check out my awesome friend's stream at http://t.co/uvbd0idwdw ! he's a fantastic artist. :),1
1,@slxstrings stop :((,0
2,@bexlittle @alisonwright8 @lindyrendy @cloud2ltd @dotnetsolutions love it thanks for sharing :),1
3,no one wants to go to ant man with me :(,0
4,@ellegowers ahaha i'm laying on my bed like whyyyyyy. the pretty wine glasses sucked me in :(,0


#2. Hapus semua link yang ada !!!!

In [18]:
df.text = df.text.str.replace(r'https?://\S+|www\.\S+','',regex=True)
df.head()

,text,label
0,please go check out my awesome friend's stream at ! he's a fantastic artist. :),1
1,@slxstrings stop :((,0
2,@bexlittle @alisonwright8 @lindyrendy @cloud2ltd @dotnetsolutions love it thanks for sharing :),1
3,no one wants to go to ant man with me :(,0
4,@ellegowers ahaha i'm laying on my bed like whyyyyyy. the pretty wine glasses sucked me in :(,0


#Hapus juga email kalau ada !

In [21]:
text = 'Silahkan hubungi saya lewat michaelsabdah@gmail.com tapi mboh-mbohan ya!'
re.sub(r'\S+@\S+','', text)

'Silahkan hubungi saya lewat  tapi mboh-mbohan ya!'

#Hapus Tanggal!

In [22]:
text = 'Libur lebaran mulai tanggal 08/04/2024 sampai tanggal 20-04-2024 bisa di perpanjang sampai 21st.04.2024'
re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}','', text)

'Libur lebaran mulai tanggal  sampai tanggal  bisa di perpanjang sampai '

#Hapus juga kalau ada HTML

In [23]:
text = """
<title>Contoh Web Sederhana</title>
<body>
<p>AEWRHALWEJFLWAEFKAF</p>
<a href= "https://uksw.edu">Alamat Web Kampus</a>
</body>

"""

In [25]:
text

'\n<title>Contoh Web Sederhana</title>\n<body>\n<p>AEWRHALWEJFLWAEFKAF</p>\n<a href= "https://uksw.edu">Alamat Web Kampus</a>\n</body>\n\n'

In [26]:
pattern = re.compile('<.*?>')
pattern.sub('',text)

'\nContoh Web Sederhana\n\nAEWRHALWEJFLWAEFKAF\nAlamat Web Kampus\n\n\n'

#Atau pakai BeautifulSoup

In [27]:
def remove_html(text):
  clean_text = BeautifulSoup(text).get_text()
  return clean_text

In [28]:
remove_html(text)

'Contoh Web Sederhana\n\nAEWRHALWEJFLWAEFKAF\nAlamat Web Kampus\n\n'